# Алгоритм новой Себестоимости для бухгалтерии

In [1]:
from tqdm import tqdm
import pandas as pd
import datetime
import numpy as np
import warnings
warnings.simplefilter(action = 'ignore', category = Warning)
import os
import seaborn as sn
import re 
import random
import calendar
import glob
import openpyxl
from openpyxl.styles import Font, PatternFill
import pyjokes
import glob
import requests 
import json
import gspread
from pydrive.auth import GoogleAuth
from sys import exit
from pydrive.drive import GoogleDrive

In [2]:
PATH = [
    
     {'path_downloads': '/Users/internetrevolution/Белодедов/Копия работы Маслов/Загрузки Маслов/',
      'path_algoritm': '/Users/internetrevolution/Белодедов/Копия работы Маслов/Алгоритмы Маслов/'} ,
     
     {'path_downloads': 'G:/Другие компьютеры/iMac/Белодедов/Копия работы Маслов/Загрузки Маслов/',
      'path_algoritm': 'G:/Другие компьютеры/iMac/Белодедов/Копия работы Маслов/Алгоритмы Маслов/'},
    
     {'path_downloads': '/Users/macbook/Library/CloudStorage/GoogleDrive-maslowdmitry92@gmail.com/Мой диск/Загрузки Маслов/',
      'path_algoritm': '/Users/macbook/Library/CloudStorage/GoogleDrive-maslowdmitry92@gmail.com/Мой диск/Алгоритмы Маслов/'} ,
     
     {'path_downloads': '/Users/dmitriymaslov/Мой диск/Загрузки Маслов/',
      'path_algoritm': '/Users/dmitriymaslov/Мой диск/Алгоритмы Маслов/'}
     
    ]
         
i = 0
while len([py for py in glob.glob(f"{PATH[i]['path_downloads']}*.xlsx")]) == 0:
    i +=1

path_downloads = PATH[i]['path_downloads']
path_algoritm = PATH[i]['path_algoritm']

print(f"Загрузки: {path_downloads}")
print(f"Папка с алгоритмом: {path_algoritm}")

Загрузки: /Users/dmitriymaslov/Мой диск/Загрузки Маслов/
Папка с алгоритмом: /Users/dmitriymaslov/Мой диск/Алгоритмы Маслов/


In [3]:
def clear_article(data, name_columns):
    data[name_columns] = data[name_columns].astype(str)
    data = data.reset_index(drop=True)
    letter_dict = {'е': 'e', 
                   'Е': 'E', 
                   'Т': 'T', 
                   'у': 'y', 
                   'о': 'o', 
                   'О': 'O', 
                   'р': 'p', 
                   'Р': 'P', 
                   'а': 'a', 
                   'А': 'A', 
                   'Н': 'H',
                   'К': 'K',
                   'х': 'x',
                   'Х': 'X',
                   'с': 'c',
                   'С': 'C',
                   'В': 'B',
                   'М': 'M'}
    for i in range(len(data)):
        if len(set(data.loc[i][name_columns]) & set(letter_dict.keys())) > 0:
            art = data.loc[i][name_columns]
            for row in art:
                if row in letter_dict.keys():
                    art = art.replace(row, letter_dict[row])
            data.loc[i, name_columns] = art
    data[name_columns] = [re.sub(r'[^a-zA-Z0-9]', '', x) for x in list(data[name_columns])]
    data = data[data[name_columns].isin([x for x in list(data[name_columns]) if len(x) > 6])==False]
    return data

In [4]:
def data_columns_sorted(data):
    dt_col_data = dt_col(data)
    col_not_dt_data = [x for x in list(data.columns) if x not in dt_col_data]
    dt_col_data_s1 = sorted([pd.to_datetime(y) for y in dt_col_data])
    dt_col_data_s2 = [z.strftime('%d-%B-%Y') for z in dt_col_data_s1]
    col = col_not_dt_data + dt_col_data_s2
    dt_col_data = None
    col_not_dt_data = None
    dt_col_data_s1 = None
    dt_col_data_s2 = None
    data = data[col]
    return data

In [5]:
def dt_col(data):
    date_col = []
    for row in data.columns:
        try:
            pd.to_datetime(row)
            date_col.append(row)
        except:
            None
    return date_col

In [6]:
path_WB = path_algoritm + 'Все файлы для ежедневной презентации/full_WB_df.xlsx'

path_ozon = path_algoritm + 'Все файлы для ежедневной презентации/full_OZON_df.xlsx'

path_yandex = path_algoritm + 'Все файлы для ежедневной презентации/full_YA_df.xlsx'

path_sber = path_algoritm + 'Все файлы для ежедневной презентации/full_SBER_df.xlsx'

path_le = path_algoritm + 'Все файлы для ежедневной презентации/full_LE_df.xlsx'



path_actual_cost_price = path_downloads + 'Себестоимость сведенная.xlsx'

path_cost_price = path_algoritm + 'ОБЩИЕ ПРИЗНАКИ ДЛЯ МЕТРИК/Себестоимость /cost_price 2.0.xlsx'

In [8]:
month_prod = 'March-2024'

In [9]:
for path in [path_WB, path_ozon, path_yandex, path_sber, path_le]:
    if path == path_WB:
        df = pd.read_excel(path,
                               dtype={'Артикул': 'str'})
        df = df[['Артикул']+dt_col(df)]\
                .groupby('Артикул').sum()\
                .reset_index(drop=False)
        df.iloc[:, 1:] = df.iloc[:, 1:].astype(int) 
    else:
        sub_df = pd.read_excel(path,
                           dtype={'Артикул': 'str'})
        sub_df = sub_df[['Артикул']+dt_col(sub_df)]\
             .groupby('Артикул').sum()\
             .reset_index(drop=False)
        sub_df.iloc[:, 1:] = sub_df.iloc[:, 1:].astype(int)   
        
        df = pd.concat([df, sub_df], ignore_index=True)
        df = df.fillna(0)
        df.iloc[:, 1:] = df.iloc[:, 1:].astype(int)
        df = df.groupby('Артикул').sum()\
               .reset_index(drop=False)
df = data_columns_sorted(df)

In [10]:
CP_name_list = ['Себестоимость с НДС', 
                'Себестоимость без НДС']

In [11]:
CP_name_list = ['Себестоимость с НДС для наценки', 
                'Себестоимость без НДС для наценки']

In [12]:
month_prod

'March-2024'

In [13]:
df

,Артикул,23-February-2020,24-February-2020,25-February-2020,26-February-2020,27-February-2020,28-February-2020,29-February-2020,01-March-2020,02-March-2020,...,26-March-2024,27-March-2024,28-March-2024,29-March-2024,30-March-2024,31-March-2024,01-April-2024,02-April-2024,03-April-2024,04-April-2024
0,16000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,16801,2,1,0,2,0,1,2,1,0,...,0,0,0,0,0,0,0,0,0,0
2,16802,2,3,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,16803,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,16804,0,0,0,1,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,Z0401,0,0,0,0,0,0,0,0,0,...,6,6,6,7,8,6,3,6,5,3
1306,Z0402,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1307,Z0500,0,0,0,0,0,0,0,0,0,...,11,4,7,4,4,2,5,4,3,3
1308,Z0501,0,0,0,0,0,0,0,0,0,...,7,11,3,2,6,13,5,0,9,4


In [14]:
start_date = datetime.datetime(2024, 3, 1)
end_date = datetime.datetime(2024, 3, 31)
res = pd.date_range(
    min(start_date, end_date),
    max(start_date, end_date)
).strftime('%d-%B-%Y').tolist()

In [15]:
res

['01-March-2024',
 '02-March-2024',
 '03-March-2024',
 '04-March-2024',
 '05-March-2024',
 '06-March-2024',
 '07-March-2024',
 '08-March-2024',
 '09-March-2024',
 '10-March-2024',
 '11-March-2024',
 '12-March-2024',
 '13-March-2024',
 '14-March-2024',
 '15-March-2024',
 '16-March-2024',
 '17-March-2024',
 '18-March-2024',
 '19-March-2024',
 '20-March-2024',
 '21-March-2024',
 '22-March-2024',
 '23-March-2024',
 '24-March-2024',
 '25-March-2024',
 '26-March-2024',
 '27-March-2024',
 '28-March-2024',
 '29-March-2024',
 '30-March-2024',
 '31-March-2024']

In [16]:
df_prod = df[['Артикул']].copy()
df_prod[f'Продажи {month_prod}'] = [df[res].iloc[x, :].sum() for x in list(df.index)]
df_prod

,Артикул,Продажи March-2024
0,16000,0
1,16801,4
2,16802,9
3,16803,2
4,16804,0
...,...,...
1305,Z0401,550
1306,Z0402,0
1307,Z0500,143
1308,Z0501,312


In [17]:
# df_prod = df[['Артикул']].copy()
# df_prod[f'Продажи {month_prod}'] = [sum(list(df.loc[y][[x for x in list(df.columns) if month_prod in x and int(x[:2]) in list(range(20, 27))]])) for y in list(df.index)]
# df_prod

In [18]:
# df_prod = df[['Артикул']].copy()
# df_prod[f'Продажи {month_prod}'] = [sum(list(df.loc[y][[x for x in list(df.columns) if month_prod in x]])) for y in list(df.index)]
# df_prod

In [19]:
df_prod

,Артикул,Продажи March-2024
0,16000,0
1,16801,4
2,16802,9
3,16803,2
4,16804,0
...,...,...
1305,Z0401,550
1306,Z0402,0
1307,Z0500,143
1308,Z0501,312


In [20]:
pd.read_excel(path_actual_cost_price,
              sheet_name='Несведенные')

,Unnamed: 0,Артикул,Наименование,Кол-во,Себестоимость с НДС,Себестоимость без НДС,Себестоимость с НДС для наценки,Себестоимость без НДС для наценки,Файл,Дата документа,Номер поставки,Документы,комментарий,Маркер себестоимости
0,0,16801,Чехол для ноутбука,200.0,330.50,273.79,330.50,273.79,TKRU4383957,2022-04-06,00БП-000220,2022-04-06 / 00БП-000220 / TKRU4383957,NaN,NaN
1,1,16801,Чехол книжка-подставка для ноутбука,210.0,243.40,204.01,243.40,204.01,Себестоимость_Фура_1_Поставка_63,2022-07-25,00БП-000487,2022-07-25 / 00БП-000487 / Себестоимость_Фура_...,NaN,NaN
2,2,16801,Чехол книжка-подставка Smart Case для iPad 9.7...,280.0,243.71,200.71,243.71,200.71,74,2022-10-18,00БП-000708,2022-10-18 / 00БП-000708 / 74,NaN,Актуальная себестоимость
3,3,16802,Чехол для ноутбука,200.0,330.50,273.79,330.50,273.79,TKRU4383957,2022-04-06,00БП-000220,2022-04-06 / 00БП-000220 / TKRU4383957,NaN,NaN
4,4,16802,Чехол книжка-подставка для ноутбука,210.0,243.40,204.01,243.40,204.01,Себестоимость_Фура_1_Поставка_63,2022-07-25,00БП-000487,2022-07-25 / 00БП-000487 / Себестоимость_Фура_...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3341,3341,Z0500,Декантер с бокалами,200.0,1091.08,939.49,1091.08,939.49,108_химки_2,2023-05-18,00БП-000463,2023-05-18 / 00БП-000463 / 108_химки_2,NaN,NaN
3342,3342,Z0500,Декантер с бокалами,400.0,1104.31,952.72,1104.31,952.72,108_химки_3,2023-05-18,00БП-000026,2023-05-18 / 00БП-000026 / 108_химки_3,NaN,NaN
3343,3343,Z0500,Декантер с бокалами,400.0,1087.62,936.03,1087.62,936.03,108_химки_4,2023-05-18,00БП-000026,2023-05-18 / 00БП-000026 / 108_химки_4,NaN,NaN
3344,3344,Z0500,Декантер с бокалами,750.0,930.79,778.90,945.68,791.36,СС_Январь_143,2024-01-19,ИРБП-000006,ГТД по импорту ИРБП-000006 от 19.01.2024 8:00:00,NaN,Актуальная себестоимость


In [21]:
CP_name_list

['Себестоимость с НДС для наценки', 'Себестоимость без НДС для наценки']

In [22]:
act_cp = pd.read_excel(path_actual_cost_price,
                       sheet_name='Несведенные',
                       usecols=['Артикул', 
                                'Кол-во', 
                                'Дата документа'] + CP_name_list,
                       dtype={'Артикул': 'str', 
                              'Себестоимость без НДС': 'float'}).dropna()
act_cp['Кол-во'] = act_cp['Кол-во'].astype(int)
act_cp = act_cp.reset_index(drop=True)
# act_cp = clear_article(act_cp, 'Артикул').sort_values('Артикул').reset_index(drop=True)
act_cp['Дата документа'] = [pd.to_datetime(x, format='%Y-%m-%d') for x in list(act_cp['Дата документа'])]
# act_cp = df.merge(act_cp, on='Артикул', how='right').fillna(0)

In [23]:
act_cp

,Артикул,Кол-во,Себестоимость с НДС для наценки,Себестоимость без НДС для наценки,Дата документа
0,16801,200,330.50,273.79,2022-04-06
1,16801,210,243.40,204.01,2022-07-25
2,16801,280,243.71,200.71,2022-10-18
3,16802,200,330.50,273.79,2022-04-06
4,16802,210,243.40,204.01,2022-07-25
...,...,...,...,...,...
3316,Z0500,200,1091.08,939.49,2023-05-18
3317,Z0500,400,1104.31,952.72,2023-05-18
3318,Z0500,400,1087.62,936.03,2023-05-18
3319,Z0500,750,945.68,791.36,2024-01-19


In [24]:
S_df = pd.DataFrame(columns=['Артикул', 
                          'Продажи с начала даты документа']
                          + list(act_cp.columns[1:])
                          + [x+' итог.' for x in CP_name_list])
art_list = sorted(list(set(act_cp['Артикул'])))
for j in range(len(art_list)):
    art = art_list[j]
    act_cp_sub = act_cp[act_cp['Артикул']==art].sort_values(by='Дата документа').reset_index(drop=True)
    if art in list(set(df['Артикул'])):
        df_sub = df[df['Артикул']==art][['Артикул']
                                        +list(df.columns[df.columns\
                                        .get_loc(act_cp_sub.loc[0]['Дата документа'].strftime('%d-%B-%Y')):])]
        df_sub['Продажи с начала даты документа'] =  df_sub.iloc[:, 1:].sum().sum()
        act_cp_sub = df_sub[['Артикул', 'Продажи с начала даты документа']]\
                     .merge(act_cp_sub, on='Артикул', how='right')
    else:
        act_cp_sub.insert(1, 'Продажи с начала даты документа', 0)
    S = act_cp_sub.loc[0]['Продажи с начала даты документа']
    R = act_cp_sub.loc[0]['Кол-во']
    if S <= R or S == 0 or act_cp_sub['Кол-во'].sum() == 0:
        for row in CP_name_list:
            act_cp_sub[row+' итог.'] = act_cp_sub.loc[0][row]
    elif S > act_cp_sub['Кол-во'].sum():
        for row in CP_name_list:
            act_cp_sub[row+' итог.'] = act_cp_sub.loc[len(act_cp_sub)-1][row]
    else:
        i = 0
        while S > R and i < len(act_cp_sub)-1:
            i = i + 1
            R = R + act_cp_sub.loc[i]['Кол-во']
        for row in CP_name_list:
            act_cp_sub[row+' итог.'] = act_cp_sub.loc[i][row]
    S_df = pd.concat([S_df, act_cp_sub], ignore_index=True)
S_df.insert(3, 'Продажи с заявленной себестоимостью', 0)


In [25]:
for art in list(set(S_df['Артикул'])):
    index_list = list(S_df[S_df['Артикул']==art].index)
    P = S_df.loc[index_list[0]]['Продажи с начала даты документа']
    for ind in index_list:      
        if S_df.loc[ind]['Кол-во'] <= P:
            S_df.loc[ind, 'Продажи с заявленной себестоимостью'] = S_df.loc[ind]['Кол-во'] 
            P = P - S_df.loc[ind]['Кол-во']
        else:
            S_df.loc[ind, 'Продажи с заявленной себестоимостью'] = P
            P = 0

In [26]:
S_df[S_df['Артикул']=='16930']

,Артикул,Продажи с начала даты документа,Кол-во,Продажи с заявленной себестоимостью,Себестоимость с НДС для наценки,Себестоимость без НДС для наценки,Дата документа,Себестоимость с НДС для наценки итог.,Себестоимость без НДС для наценки итог.
15,16930,3540,1008,1008,303.94,241.38,2022-10-18,358.69,294.48
16,16930,3540,504,504,364.14,300.99,2023-03-21,358.69,294.48
17,16930,3540,1008,1008,355.83,296.74,2023-05-12,358.69,294.48
18,16930,3540,840,840,358.36,299.27,2023-05-12,358.69,294.48
19,16930,3540,168,168,361.52,302.43,2023-05-12,358.69,294.48
20,16930,3540,984,12,358.69,294.48,2023-08-05,358.69,294.48
21,16930,3540,1224,0,376.34,314.56,2024-02-12,358.69,294.48


In [27]:
S_df = S_df.merge(df_prod, on='Артикул', how='left')
S_df = S_df[list(S_df.columns)[:4]+[list(S_df.columns)[-1]]+list(S_df.columns)[4:-1]]

In [28]:
month_prod

'March-2024'

In [29]:
S_df.insert(5, f'Продажи {month_prod} по себестоимостям', 0)

In [30]:
S_df

,Артикул,Продажи с начала даты документа,Кол-во,Продажи с заявленной себестоимостью,Продажи March-2024,Продажи March-2024 по себестоимостям,Себестоимость с НДС для наценки,Себестоимость без НДС для наценки,Дата документа,Себестоимость с НДС для наценки итог.,Себестоимость без НДС для наценки итог.
0,16801,873,200,200,4.0,0,330.50,273.79,2022-04-06,243.71,200.71
1,16801,873,210,210,4.0,0,243.40,204.01,2022-07-25,243.71,200.71
2,16801,873,280,280,4.0,0,243.71,200.71,2022-10-18,243.71,200.71
3,16802,1239,200,200,9.0,0,330.50,273.79,2022-04-06,243.71,200.71
4,16802,1239,210,210,9.0,0,243.40,204.01,2022-07-25,243.71,200.71
...,...,...,...,...,...,...,...,...,...,...,...
3316,Z0500,1944,200,0,143.0,0,1091.08,939.49,2023-05-18,1089.52,937.93
3317,Z0500,1944,400,0,143.0,0,1104.31,952.72,2023-05-18,1089.52,937.93
3318,Z0500,1944,400,0,143.0,0,1087.62,936.03,2023-05-18,1089.52,937.93
3319,Z0500,1944,750,0,143.0,0,945.68,791.36,2024-01-19,1089.52,937.93


In [31]:
for art in list(set(S_df['Артикул'])):
    index_list = list(S_df[S_df['Артикул']==art].index)[::-1]
    P = S_df.loc[index_list[0]][f'Продажи {month_prod}']
    for ind in index_list:      
        if S_df.loc[ind]['Продажи с заявленной себестоимостью'] >= P:
            S_df.loc[ind, f'Продажи {month_prod} по себестоимостям'] = P
            P = 0
        else:
            S_df.loc[ind, f'Продажи {month_prod} по себестоимостям'] = S_df.loc[ind, 'Продажи с заявленной себестоимостью']
            P = P - S_df.loc[ind, 'Продажи с заявленной себестоимостью']

In [32]:
S_df[S_df['Артикул']=='16980']

,Артикул,Продажи с начала даты документа,Кол-во,Продажи с заявленной себестоимостью,Продажи March-2024,Продажи March-2024 по себестоимостям,Себестоимость с НДС для наценки,Себестоимость без НДС для наценки,Дата документа,Себестоимость с НДС для наценки итог.,Себестоимость без НДС для наценки итог.
35,16980,3707,1500,1500,323.0,0,345.22,292.33,2023-05-26,279.82,232.85
36,16980,3707,1500,1500,323.0,0,349.75,296.86,2023-05-26,279.82,232.85
37,16980,3707,1584,707,323.0,323,279.82,232.85,2024-01-03,279.82,232.85


In [33]:
month_prod

'March-2024'

In [34]:
S_df_final = S_df[['Артикул', f'Продажи {month_prod} по себестоимостям']+CP_name_list + [x+' итог.' for x in CP_name_list]][S_df[f'Продажи {month_prod} по себестоимостям']!=0].reset_index(drop=True)

In [35]:
S_df_final.to_excel(path_downloads
          + f'Себестоимость за_{month_prod}.xlsx', index=False)

In [36]:
S_df.to_excel(path_downloads + 'Себестоимость итоговая для бухгалтерии.xlsx', index=False)
pd.concat([S_df[['Артикул']+[x+' итог.' for x in CP_name_list]].drop_duplicates().reset_index(drop=True),
           pd.read_excel(path_cost_price,
                         dtype={'Артикул': 'str'}).query(f"Артикул not in {list(set(S_df['Артикул']))}")],
          ignore_index=True)\
.to_excel(path_downloads
          + f'Себестоимость итоговая_{datetime.datetime.now().strftime("%d.%m.%y")}.xlsx', index=False)

In [37]:
exit()

SystemExit: 

In [38]:
r = random.choice(art_list)
display(act_cp[act_cp['Артикул']==r])

print(df[df['Артикул']==r][['Артикул']
                                        +list(df.columns[df.columns\
                                        .get_loc(act_cp[act_cp['Артикул']==r].reset_index(drop=True).loc[0]['Дата документа'].strftime('%d-%B-%Y')):])]\
.iloc[:, 1:].sum().sum())

display(S_df[S_df['Артикул']==r])

,Артикул,Кол-во,Себестоимость с НДС для наценки,Себестоимость без НДС для наценки,Дата документа
2260,D0055,3000,83.77,69.84,2023-12-22


449


,Артикул,Продажи с начала даты документа,Кол-во,Продажи с заявленной себестоимостью,Продажи March-2024,Продажи March-2024 по себестоимостям,Себестоимость с НДС для наценки,Себестоимость без НДС для наценки,Дата документа,Себестоимость с НДС для наценки итог.,Себестоимость без НДС для наценки итог.
2260,D0055,449,3000,449,280.0,280,83.77,69.84,2023-12-22,83.77,69.84


In [39]:
16802

16802

In [40]:
S_df[S_df['Артикул']=='16802']

,Артикул,Продажи с начала даты документа,Кол-во,Продажи с заявленной себестоимостью,Продажи March-2024,Продажи March-2024 по себестоимостям,Себестоимость с НДС для наценки,Себестоимость без НДС для наценки,Дата документа,Себестоимость с НДС для наценки итог.,Себестоимость без НДС для наценки итог.
3,16802,1239,200,200,9.0,0,330.50,273.79,2022-04-06,243.71,200.71
4,16802,1239,210,210,9.0,0,243.40,204.01,2022-07-25,243.71,200.71
5,16802,1239,630,630,9.0,9,243.71,200.71,2022-10-18,243.71,200.71
